# Wikipedia Edits Data Pipeline

This notebook demonstrates a simple ETL pipeline that:
1. Extracts data from the Wikipedia REST API
2. Transforms it to JSON Lines format
3. Uploads directly to S3

In [42]:
# Set your username here - use it consistently across all resources
USERNAME = "nahian"

## Setup and Imports

In [43]:
import datetime
import json

import boto3
import requests

## Extract: Retrieve Data from Wikipedia API

We use the Wikimedia Analytics API to fetch the most edited pages for a specific date. The API returns JSON with page titles and edit counts.

**API Documentation:** https://doc.wikimedia.org/generated-data-platform/aqs/analytics-api/reference/edits.html

In [44]:
# Try different dates to see how the data changes
DATE_PARAM = "2025-11-25"

date = datetime.datetime.strptime(DATE_PARAM, "%Y-%m-%d")

# Construct the API URL
url = f"https://wikimedia.org/api/rest_v1/metrics/pageviews/top/en.wikipedia.org/all-access/{date.strftime('%Y/%m/%d')}"
print(f"Requesting REST API URL: {url}")

# Make the API request
wiki_server_response = requests.get(url, headers={"User-Agent": "curl/7.68.0"})
wiki_response_status = wiki_server_response.status_code
wiki_response_body = wiki_server_response.text

print(f"Wikipedia REST API Response body: {wiki_response_body[:500]}...")
print(f"Wikipedia REST API Response Code: {wiki_response_status}")

# Validate response
if wiki_response_status != 200:
    raise Exception(f"Received non-OK status code from Wiki Server: {wiki_response_status}")
print(f"Successfully retrieved Wikipedia data, content-length: {len(wiki_response_body)}")

Requesting REST API URL: https://wikimedia.org/api/rest_v1/metrics/pageviews/top/en.wikipedia.org/all-access/2025/11/25
Wikipedia REST API Response body: {"items":[{"project":"en.wikipedia","access":"all-access","year":"2025","month":"11","day":"25","articles":[{"article":"Main_Page","views":6343624,"rank":1},{"article":"Special:Search","views":867370,"rank":2},{"article":"Dharmendra","views":386721,"rank":3},{"article":"Sawyer_Sweeten","views":314428,"rank":4},{"article":"Google_Chrome","views":308826,"rank":5},{"article":"Wikipedia:Featured_pictures","views":259967,"rank":6},{"article":"Richard_Branson","views":160194,"rank":7},{"article":"Wick...
Wikipedia REST API Response Code: 200
Successfully retrieved Wikipedia data, content-length: 55589


## Transform: Process Raw Data into JSON Lines

Convert the raw API response into a structured JSON Lines format suitable for analytics. Each line is a valid JSON object representing one page's edit statistics.

In [45]:
# Parse the API response and extract top edits
wiki_response_parsed = wiki_server_response.json()
top_views = wiki_response_parsed["items"][0]["articles"]

# Transform to JSON Lines format
current_time = datetime.datetime.now(datetime.timezone.utc)
json_lines = ""
for page in top_views[:5]:
    record = {
        "title": page["article"],
        "views": page["views"],
        "rank": page["rank"],
        "date": date.strftime("%Y-%m-%d"),
        "retrieved_at": current_time.replace(tzinfo=None).isoformat(),
    }
    json_lines += json.dumps(record) + "\n"

print(f"Transformed {len(top_views)} records to JSON Lines")
print(f"First few lines:\n{json_lines[:500]}...")

Transformed 1000 records to JSON Lines
First few lines:
{"title": "Main_Page", "views": 6343624, "rank": 1, "date": "2025-11-25", "retrieved_at": "2025-12-19T18:34:47.056078"}
{"title": "Special:Search", "views": 867370, "rank": 2, "date": "2025-11-25", "retrieved_at": "2025-12-19T18:34:47.056078"}
{"title": "Dharmendra", "views": 386721, "rank": 3, "date": "2025-11-25", "retrieved_at": "2025-12-19T18:34:47.056078"}
{"title": "Sawyer_Sweeten", "views": 314428, "rank": 4, "date": "2025-11-25", "retrieved_at": "2025-12-19T18:34:47.056078"}
{"title": "G...


---
## Lab 1: Create an S3 Bucket

**Task:** Create an S3 bucket for the Wikipedia data pipeline.

**Requirements:**
- Bucket name: `<username>-wikidata` (use your USERNAME from above)
- Create the bucket if it doesn't exist

**Documentation:** [create_bucket](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/client/create_bucket.html)

In [46]:
S3_WIKI_BUCKET = "nahian-wikidata"
s3 = boto3.client("s3")

bucket_names = [bucket["Name"] for bucket in s3.list_buckets()["Buckets"]]
if S3_WIKI_BUCKET not in bucket_names:
    # LAB 1: Create the bucket if it doesn't exist
    # YOUR SOLUTION COMES HERE =========================
    default_region = "eu-west-1"
    bucket_configuration = {"LocationConstraint": default_region}
    response = s3.create_bucket(Bucket= S3_WIKI_BUCKET, CreateBucketConfiguration=bucket_configuration)
    # ==================================================
    print(f"Created new bucket: {S3_WIKI_BUCKET}")
else:
    print(f"Using existing bucket: {S3_WIKI_BUCKET}")


Using existing bucket: nahian-wikidata


In [47]:
# Test Lab 1
assert USERNAME != "nahian-wikipedia", "Please set your USERNAME at the top of the notebook"
assert S3_WIKI_BUCKET.endswith("-wikidata"), "Bucket name must end with '-wikidata'"

try:
    s3.head_bucket(Bucket=S3_WIKI_BUCKET)
    print(f"Bucket {S3_WIKI_BUCKET} exists!")
except Exception as e:
    print(f"Bucket {S3_WIKI_BUCKET} not found: {e}")
    raise

Bucket nahian-wikidata exists!


---
## Lab 2: Upload JSON Lines to S3

**Task:** Upload the `json_lines` data directly to S3 (no local file!).

**Requirements:**
- Use `s3.put_object()` to upload the data directly
- Place the file under `raw-edits/` prefix in S3
- File name: `raw-edits-YYYY-MM-DD.json` (use the date from `DATE_PARAM`)

**Example S3 path:** `s3://johndoe-wikidata/raw-edits/raw-edits-2025-11-25.json`

**Documentation:** [put_object](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/client/put_object.html)

In [48]:
# LAB 2: Upload json_lines directly to S3

s3_key = f"raw-views/raw-views-{DATE_PARAM}.json"

# Upload the json_lines string directly to S3
s3.put_object(
    Bucket=S3_WIKI_BUCKET,
    Key=s3_key,
    Body=json_lines,
    ContentType="application/json"
)


{'ResponseMetadata': {'RequestId': '0NYM8AMJMFQHNVVH',
  'HostId': '2cemRRgw/ZLEUa6yKHkNVI39nDa6ZG+5YCW1q7uF38tjIJ90c4V5lAhZlkO5Rv2+FSs6vQ0jHEs=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '2cemRRgw/ZLEUa6yKHkNVI39nDa6ZG+5YCW1q7uF38tjIJ90c4V5lAhZlkO5Rv2+FSs6vQ0jHEs=',
   'x-amz-request-id': '0NYM8AMJMFQHNVVH',
   'date': 'Fri, 19 Dec 2025 18:34:48 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"4130b2195513992670e90decb8f7db74"',
   'x-amz-checksum-crc32': 'fh187A==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"4130b2195513992670e90decb8f7db74"',
 'ChecksumCRC32': 'fh187A==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}

In [49]:
# Test Lab 2
expected_key = f"raw-views/raw-views-{date.strftime('%Y-%m-%d')}.json"

try:
    s3.head_object(Bucket=S3_WIKI_BUCKET, Key=expected_key)
    print(f"File uploaded successfully to s3://{S3_WIKI_BUCKET}/{expected_key}")
except Exception as e:
    print(f"File not found at s3://{S3_WIKI_BUCKET}/{expected_key}")
    raise

File uploaded successfully to s3://nahian-wikidata/raw-views/raw-views-2025-11-25.json
